In [1]:
import numpy

In [206]:
def blockproc_reshape_v1(
    array: numpy.ndarray,
    block_size: numpy.ndarray,
    order: str
) -> numpy.ndarray:

    nb_rows, nb_cols = array.shape
    res = None

    step_r, step_c = block_size

    for j in range(0, nb_cols, step_c):

        column = None
        
        for i in range(0, nb_rows, step_r):
            
            bloc = numpy.reshape(
                array[i:i+step_r, j:j+step_c], 
                (step_r*step_c, 1),
                order=order
            )
            if column is None:
                column = numpy.copy(bloc)
            else:
                column = numpy.vstack([ column, bloc ])

        if res is None:
            res = numpy.copy(column)
        else:
            res = numpy.hstack([ res, column ])

    return res

def blockproc_reshape_v2(
    array: numpy.ndarray,
    block_size: numpy.ndarray,
    order: str
) -> numpy.ndarray:

    nb_rows, nb_cols = array.shape
    step_r, step_c = block_size
    res = numpy.empty(shape = (nb_rows // step_r, nb_cols // step_c), dtype=numpy.ndarray)

    for i in range(0, res.shape[0]):
        for j in range(0, res.shape[1]):
     
            res[i, j] = numpy.reshape(
                array[
                    i*step_r:(i+1)*step_r, 
                    j*step_c:(j+1)*step_c
                ], 
                (step_r*step_c, 1),
                order=order
            )

    return numpy.hstack([ numpy.vstack(res[:, j]) for j in range(0, res.shape[1])])

def blockproc_reshape_v3(
    array: numpy.ndarray,
    block_size: numpy.ndarray,
    order: str
) -> numpy.ndarray:

    nb_rows, nb_cols = array.shape
    step_r, step_c = block_size
    res = numpy.empty(shape = (nb_rows // step_r, nb_cols // step_c), dtype=numpy.ndarray)

    for i in range(0, res.shape[0]):
        for j in range(0, res.shape[1]):
     
            res[i, j] = numpy.reshape(
                array[
                    i*step_r:(i+1)*step_r, 
                    j*step_c:(j+1)*step_c
                ], 
                (step_r*step_c, 1),
                order=order
            )

    out = numpy.zeros(shape=(step_r*step_c*res.shape[0], res.shape[1]))
    for i in range(0, res.shape[0]):
        for j in range(0, res.shape[1]):
            for k in range(0, step_r*step_c):
                out[i*(step_r*step_c)+k, j] = res[i, j][k]

    return out


def blockproc_reshape3D(
    array: numpy.ndarray,
    block_size: numpy.ndarray,
    order: str
) -> numpy.ndarray:

    nb1, nb2, nb3 = array.shape
    step1, step2, step3 = block_size
    res = numpy.empty(
        shape = [
            nb1 // step1, 
            nb2 // step2,
            nb3 // step3,
        ], 
        dtype=numpy.ndarray
    )

    for i in range(0, res.shape[0]):
        for j in range(0, res.shape[1]):
            for k in range(0, res.shape[2]):

                res[i, j, k] = numpy.reshape(
                    array[
                        i*step1:(i+1)*step1, 
                        j*step2:(j+1)*step2,
                        k*step3:(k+1)*step3
                    ], 
                    (step1*step2*step3, 1),
                    order=order
                )

    out = numpy.zeros(shape=(step1*step2*step3*res.shape[0], res.shape[1], res.shape[2]))
    for i in range(0, res.shape[0]):
        for j in range(0, res.shape[1]):
            for k in range(0, res.shape[2]):
                for l in range(0, step1*step2*step3):
                    out[i*(step1*step2*step3)+l, j, k] = res[i, j, k][l]

    return out

   

In [207]:
mat4x4 = numpy.reshape(numpy.arange(0, 16), newshape=(4, 4))
mat4x4

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [208]:
blockproc_reshape_v1(mat4x4, block_size=(2, 2), order='F')

array([[ 0,  2],
       [ 4,  6],
       [ 1,  3],
       [ 5,  7],
       [ 8, 10],
       [12, 14],
       [ 9, 11],
       [13, 15]])

In [209]:
blockproc_reshape_v2(mat4x4, block_size=(2, 2), order='F')

array([[ 0,  2],
       [ 4,  6],
       [ 1,  3],
       [ 5,  7],
       [ 8, 10],
       [12, 14],
       [ 9, 11],
       [13, 15]])

In [210]:
blockproc_reshape_v3(mat4x4, block_size=(2, 2), order='F')

array([[ 0.,  2.],
       [ 4.,  6.],
       [ 1.,  3.],
       [ 5.,  7.],
       [ 8., 10.],
       [12., 14.],
       [ 9., 11.],
       [13., 15.]])

In [199]:
mat4x4x4 = numpy.reshape(numpy.arange(0, 4**3), newshape=(4, 4, 4))
mat4x4x4

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23],
        [24, 25, 26, 27],
        [28, 29, 30, 31]],

       [[32, 33, 34, 35],
        [36, 37, 38, 39],
        [40, 41, 42, 43],
        [44, 45, 46, 47]],

       [[48, 49, 50, 51],
        [52, 53, 54, 55],
        [56, 57, 58, 59],
        [60, 61, 62, 63]]])

In [211]:
r = blockproc_reshape3D(mat4x4x4,  block_size=(2, 2, 2), order='F')
r[0]

array([[ 0.,  2.],
       [ 8., 10.]])

In [212]:
r.shape

(16, 2, 2)

In [213]:
x1 = numpy.reshape(r, newshape=(2**3, 8), order='F')
x1

array([[ 0., 32.,  8., 40.,  2., 34., 10., 42.],
       [16., 48., 24., 56., 18., 50., 26., 58.],
       [ 4., 36., 12., 44.,  6., 38., 14., 46.],
       [20., 52., 28., 60., 22., 54., 30., 62.],
       [ 1., 33.,  9., 41.,  3., 35., 11., 43.],
       [17., 49., 25., 57., 19., 51., 27., 59.],
       [ 5., 37., 13., 45.,  7., 39., 15., 47.],
       [21., 53., 29., 61., 23., 55., 31., 63.]])

In [214]:
x1 = numpy.sum(x1, axis=1)
x1

array([168., 296., 200., 328., 176., 304., 208., 336.])

In [215]:
x = numpy.reshape(x1, newshape=(2, 2, 2), order='F')
x

array([[[168., 176.],
        [200., 208.]],

       [[296., 304.],
        [328., 336.]]])

In [216]:
x.shape

(2, 2, 2)